In [1]:
%env GOOGLE_APPLICATION_CREDENTIALS=/tf/mounted/bl-scale_key.json
!head -n 3 $GOOGLE_APPLICATION_CREDENTIALS

env: GOOGLE_APPLICATION_CREDENTIALS=/tf/mounted/bl-scale_key.json
{
  "type": "service_account",
  "project_id": "breakthrough-listen-sandbox",


In [2]:
from google.cloud import storage
import os
from tqdm import tqdm
from multiprocessing import Pool, current_process
import time

In [31]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    
def upload_dir(bucket_name, dir_name):
    start = time.time()
    paths = []
    for root, dirs, files in os.walk(os.path.join(dir_name, "filtered")):
        paths_temp = filter(lambda x: x.endswith(".png"), files)
        paths_temp = [os.path.join(root, path) for path in paths_temp]
        paths.extend(paths_temp)
        
    print(f"Found {len(paths)} stamps in {dir_name}, uploading")
        
    global upload_file
    def upload_file(path):
        upload_blob("bl-scale", path, "/".join(path.split("/")[-4:]))
        
    start_png = time.time()
    with Pool(os.cpu_count()) as p:
        p.map(upload_file, paths)
    del upload_file
        
    end_png = time.time()
    
    print(f"Uploaded {len(paths)} stamps in {end_png-start_png} seconds")
    print(f"Average upload time: {(end_png-start_png) / len(paths)}")
    
    print(f"Uploading metadata")
    
    upload_blob("bl-scale", os.path.join(dir_name, "header.pkl"),
                os.path.join(os.path.basename(dir_name), "header.pkl"))
    upload_blob("bl-scale", os.path.join(dir_name, "info_df.pkl"),
                os.path.join(os.path.basename(dir_name), "info_df.pkl"))
    
    end = time.time()
    
    print(f"Metadata uploaded")
    print(f"Outputs for {os.path.basename(dir_name)} uploaded in {end-start} seconds")
    

In [4]:
!ls /tf/data/S-band-high/

GBT_58010_50176_HIP61317_fine	GBT_58210_40853_HIP106147_fine
GBT_58014_69579_HIP77629_fine	GBT_58210_41185_HIP105761_fine
GBT_58110_60123_HIP91926_fine	GBT_58307_26947_J1935+1616_fine
GBT_58202_60970_B0329+54_fine	GBT_58452_79191_HIP115687_fine
GBT_58210_37805_HIP103730_fine	used
GBT_58210_39862_HIP105504_fine


In [5]:
paths = []
for root, dirs, files in tqdm(os.walk(os.path.join("/tf/data/S-band-high/", "GBT_58014_69579_HIP77629_fine/filtered"))):
    paths_temp = filter(lambda x: x.endswith(".png"), files)
    paths_temp = [os.path.join(root, path) for path in paths_temp]
    paths.extend(paths_temp)
print(len(paths))

12it [00:00, 32.99it/s]

82137


In [21]:
upload = os.listdir("/tf/data/S-band-high/")

In [27]:
upload.pop(5)

'GBT_58014_69579_HIP77629_fine'

In [28]:
upload

['GBT_58210_40853_HIP106147_fine',
 'GBT_58210_39862_HIP105504_fine',
 'GBT_58210_37805_HIP103730_fine',
 'GBT_58452_79191_HIP115687_fine',
 'GBT_58210_41185_HIP105761_fine',
 'GBT_58202_60970_B0329+54_fine',
 'GBT_58307_26947_J1935+1616_fine']

In [32]:
for d in tqdm(upload):
    upload_dir("bl-scale", os.path.join("/tf/data/S-band-high/", d))

  0%|          | 0/7 [00:00<?, ?it/s]

Found 77098 stamps in /tf/data/S-band-high/GBT_58210_40853_HIP106147_fine, uploading
Uploaded 77098 stamps in 990.0112552642822 seconds
Average upload time: 0.012840946007215262
Uploading metadata


 14%|█▍        | 1/7 [16:31<1:39:06, 991.06s/it]

Metadata uploaded
Outputs for GBT_58210_40853_HIP106147_fine uploaded in 991.0523264408112 seconds
Found 65335 stamps in /tf/data/S-band-high/GBT_58210_39862_HIP105504_fine, uploading
Uploaded 65335 stamps in 930.648256778717 seconds
Average upload time: 0.014244252801388491
Uploading metadata


 29%|██▊       | 2/7 [32:03<1:21:07, 973.58s/it]

...

Metadata uploaded
Outputs for GBT_58210_41185_HIP105761_fine uploaded in 964.8655440807343 seconds
Found 48472 stamps in /tf/data/S-band-high/GBT_58202_60970_B0329+54_fine, uploading
Uploaded 48472 stamps in 617.5165219306946 seconds
Average upload time: 0.012739654273202974
Uploading metadata


 86%|████████▌ | 6/7 [1:30:34<14:23, 863.36s/it]

Metadata uploaded
Outputs for GBT_58202_60970_B0329+54_fine uploaded in 619.3381230831146 seconds
Found 66973 stamps in /tf/data/S-band-high/GBT_58307_26947_J1935+1616_fine, uploading
Uploaded 66973 stamps in 840.3162631988525 seconds
Average upload time: 0.012547090069115203
Uploading metadata


100%|██████████| 7/7 [1:44:37<00:00, 857.15s/it]

Metadata uploaded
Outputs for GBT_58307_26947_J1935+1616_fine uploaded in 842.6606967449188 seconds


In [33]:
7*70000*10 / 1024

4785.15625

In [1]:
import h5py

In [2]:
hf = h5py.File("/tf/data/S-band-high/used/GBT_58307_26947_J1935+1616_fine.h5")

In [3]:
arr = hf["data"][:]

OSError: Can't read data (can't open directory: /usr/local/hdf5/lib/plugin)